<a href="https://colab.research.google.com/github/eduzortea/SmartPill_Alura_Aula_4/blob/main/SmartPill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar SDK

In [ ]:
pip install -q -U google-generativeai

In [ ]:
!pip install virtualenv


In [ ]:
!virtualenv biblioteca1

In [ ]:
!source biblioteca1/bin/activate


biblioteca de autenticacao a servicos do google:

In [ ]:
pip install --upgrade google-auth==2.27.0


In [ ]:
pip install reportlab

In [ ]:
pip install qrcode

Inicializar a IA


In [109]:
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors
from datetime import datetime, timedelta
import qrcode
from google.colab import files
import google.generativeai as genai
from google.colab import userdata

# Obter a chave de API e configurar o genai
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

system_instruction = "Gere uma tabela de programação de medicamentos considerando as informações fornecidas. Cada linha na tabela deve representar um medicamento com as colunas correspondendo aos dias de tratamento, começando pela data de início especificada. Para cada medicamento, ajuste os horários de dose para coincidir com as refeições se necessário e não inclua horários de refeição já passados no dia de início. Use o formato de saída abaixo para a tabela:\n\nFormato da tabela:\n- A primeira coluna deve listar o nome do medicamento.\n- As colunas subsequentes devem representar cada dia de tratamento, iniciando pela data de início, com os horários das doses listados.\n\nExemplo de entrada:\n- Nome do Medicamento: Aspirina, Frequência: 8h, Com Refeições: Sim, Dias: 10, Início: 20240510\n- Nome do Medicamento: Ibuprofeno, Frequência: 12h, Com Refeições: Não, Dias: 15, Início: 20240510\n\nExpectativa de saída:\n| Medicamento | 10/05/2024 | 11/05/2024 | 12/05/2024 | ... |\n|-------------|------------|------------|------------|-----|\n| Aspirina    | 08:00, 12:00, 19:00 | 08:00, 12:00, 19:00 | 08:00, 12:00, 19:00 | ... |\n| Ibuprofeno  | 00:00, 12:00 | 00:00, 12:00 | 00:00, 12:00 | ... |"

# Nome do modelo do GenAI
model = genai.GenerativeModel(model_name="gemini-1.0", system_instruction=system_instruction)

# Solicitar informações do usuário
quantidade = input("Quantos medicamentos deseja incluir no plano:")
medicamento = input("Nome do primeiro Medicamento: ")
frequencia = input("Frequência (em horas): ")
refeicoes = input("Com refeições (Sim ou Não): ")
dias = input("Dias de Tratamento: ")
inicio = input("Data de Início (no formato AAAA-MM-DD): ")

# Gerar a programação usando o modelo GenAI
prompt_parts = [
    f"Quantos medicamentos deseja incluir no plano: {quantidade}",
    f"Nome do primeiro Medicamento: {medicamento}",
    f"Frequência: {frequencia}",
    f"Com Refeições: {refeicoes}",
    f"Dias: {dias}",
    f"Início: {inicio}",
]

response = model.generate_content(prompt_parts)

print(response.text)


Quantos medicamentos deseja incluir no plano:1
Nome do primeiro Medicamento: hjlj
Frequência (em horas): 8h
Com refeições (Sim ou Não): sim
Dias de Tratamento: 5
Data de Início (no formato AAAA-MM-DD): 2024-05-10


NotFound: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.0 is not found for API version v1beta, or is not supported for GenerateContent. Call ListModels to see the list of available models and their supported methods.